# Decision Tree Visualizations
This notebook demonstrates key visualization techniques for decision trees.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from dmba import plotDecisionTree, classificationSummary
import dmba

# Load the data
mower_df = dmba.load_data('RidingMowers.csv')

# Create and fit the model
X = mower_df.drop(columns=['Ownership'])
y = mower_df['Ownership']
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X, y)

# 1. Basic Decision Tree Visualization
plt.figure(figsize=(10, 6))
plotDecisionTree(tree, feature_names=X.columns, class_names=tree.classes_)
plt.title('Basic Decision Tree Structure')
plt.show()

# 2. Feature Space Partitioning
plt.figure(figsize=(8, 6))
plt.scatter(mower_df[mower_df['Ownership']=='Owner']['Income'], 
           mower_df[mower_df['Ownership']=='Owner']['Lot_Size'],
           label='Owner')
plt.scatter(mower_df[mower_df['Ownership']=='Nonowner']['Income'],
           mower_df[mower_df['Ownership']=='Nonowner']['Lot_Size'],
           label='Nonowner')
plt.xlabel('Income ($000s)')
plt.ylabel('Lot Size (000s sqft)')
plt.title('Decision Boundaries in Feature Space')
plt.legend()
plt.show()

# 3. Feature Importance Plot
importances = tree.feature_importances_
features = X.columns
plt.figure(figsize=(8, 4))
plt.barh(features, importances)
plt.xlabel('Feature Importance')
plt.title('Variable Importance in Decision Tree')
plt.show()